# _PREDICTIONS HEART DISEASE DATA SET_

<hr style="height:2px;">

[Official Link to the Data Set](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)

<br>

## Sumary
- [Preparing the ambient](#Preparing-the-ambient)

<hr>

## _Preparing the ambient_

First, we need prepare the packages that we will use.
The packages are "DataFrames", "MLJ", "MLJLinearModels", "Plots".

In [67]:
using DataFrames, MLJ, MLJLinearModels, Plots, CSV, DecisionTree, MLJFlux, Flux, LIBSVM, ScikitLearn, MLJScikitLearnInterface;

┌ Info: Precompiling MLJScikitLearnInterface [5ae90465-5518-4432-b9d2-8a1def2f0cab]
└ @ Base loading.jl:1260
┌ Info: Installing sklearn via the Conda scikit-learn package...
└ @ PyCall C:\Users\yancf\.julia\packages\PyCall\zqDXB\src\PyCall.jl:697
┌ Info: Running `conda install -y scikit-learn` in root environment
└ @ Conda C:\Users\yancf\.julia\packages\Conda\3rPhK\src\Conda.jl:113


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yancf\.julia\conda\3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    icc_rt-2019.0.0            |       h0cc432a_1         6.0 MB
    joblib-0.16.0              |             py_0         210 KB
    scikit-learn-0.23.1        |   py38h25d0782_0         4.7 MB
    scipy-1.5.0                |   py38h9439919_0        12.0 MB
    threadpoolctl-2.1.0        |     pyh5ca1d4c_0          17 KB
    ------------------------------------------------------------
                                           Total:        22.9 MB

The following NEW packages will be INSTALLED:

  icc_rt             pkgs/main/win-64::icc_rt-2019.0.0-h0cc432a_1
  joblib             pkgs/main/noarch::joblib-0.16.0-py_0
  scikit-learn       pkgs/main/win-64::scikit-learn-0.23.1-py38h25d

If some of the packages is not instaled in your computer just use:
```julia
    using Pkg;
    Pkg.add("PackageName");
```   
<br>

[Back to the top](#Sumary)

In [41]:
function trainer(LRC_model, df::DataFrame, train::Array{Int, 1}, test::Array{Int, 1})
    
    # X and Y
    y, X = unpack(df, ==(:target), colname -> true);
    
    # Training
    LRC = machine(LRC_model, X, categorical(y));
    MLJ.fit!(LRC, rows=train);

    return LRC
end

trainer (generic function with 1 method)

In [5]:
struct Evaluation
    Accuracy::Float64
    Recall::Float64
    Precision::Float64
    F1::Float64
    MCC::Float64
    ConfusionMatrix
end

In [6]:
function _evaluation(LRC, df, test; binary=false)
    # X and Y
    y, X = unpack(df, ==(:target), colname -> true);
    
    ŷ = MLJ.predict(LRC, X[test,:]);
    result = mode.(ŷ);
    
    # Accuracy
    _accuracy = accuracy(result, categorical(y[test]));
    
    
    # Confusion Matrix
    _confusion_matrix = MLJ.confusion_matrix(result, categorical(y[test]));
    
    TN = _confusion_matrix[1];
    FN = _confusion_matrix[2];
    FP = _confusion_matrix[3];
    TP = _confusion_matrix[4];
    
    # Recall
    _recall = TP / (TP + FN)
    _precision = TP / (TP + FP)
    _f1 = (2 * _precision) / (_precision + _recall)
    
    _mcc = 0
    if(binary)
        _mcc = (TP * TN - FP * FN) / ((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))^(1/2)
    end
    
    return Evaluation(_accuracy, _recall, _precision, _f1, _mcc, _confusion_matrix)
end

_evaluation (generic function with 1 method)

# DataFrame

In [7]:
cleveland = DataFrame(CSV.File("Data/AnalysisData/AnalysisData.data"))

# Categorical

cleveland.sex = categorical(cleveland.sex);
cleveland.cp = categorical(cleveland.cp);
cleveland.fbs = categorical(cleveland.fbs);
cleveland.restecg = categorical(cleveland.restecg);
cleveland.exang = categorical(cleveland.exang);
cleveland.slope = categorical(cleveland.slope);
cleveland.thal = categorical(cleveland.thal);
cleveland.target = categorical(cleveland.target);



In [8]:
 y, X = unpack(cleveland, ==(:target), colname -> true);
train, test = partition(eachindex(y), 0.7, stratify=y);

# Logistic Regression

In [9]:
LRC_model =  MLJLinearModels.LogisticClassifier();
LRC = trainer(LRC_model, cleveland, train, test);
__evaLRC = _evaluation(LRC, cleveland, test; binary=true);

┌ Warning: The scitype of `X`, in `machine(model, X, ...)` 
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:73
┌ Info: Training Machine{LogisticClassifier} @645.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:317
┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\measures\confusion_matrix.jl:83


In [24]:
__evaLRC.ConfusionMatrix

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     44      │     12      │
├─────────────┼─────────────┼─────────────┤
│      1      │      4      │     29      │
└─────────────┴─────────────┴─────────────┘


In [10]:
println("Accuracy: $(round(__evaLRC.Accuracy, digits=2))")
println("Recall: $(round(__evaLRC.Recall, digits=2))")
println("Precision: $(round(__evaLRC.Precision, digits=2))")
println("F1: $(round(__evaLRC.F1, digits=2))")
println("MCC: $(round(__evaLRC.MCC, digits=2))")

Accuracy: 0.82
Recall: 0.88
Precision: 0.71
F1: 0.89
MCC: 0.64


# Tree

In [11]:
Tree_model = @load DecisionTreeClassifier verbosity=1
Tree = trainer(Tree_model, cleveland, train, test);
__evaTree = _evaluation(Tree, cleveland, test; binary=true);

┌ Info: Model code for DecisionTreeClassifier already loaded
└ @ MLJModels C:\Users\yancf\.julia\packages\MLJModels\5DFoi\src\loading.jl:54
┌ Warning: The scitype of `X`, in `machine(model, X, ...)` 
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:73
┌ Info: Training Machine{DecisionTreeClassifier} @460.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:317
┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\measures\confusion_matrix.jl:83


In [27]:
__evaTree.ConfusionMatrix

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     39      │     15      │
├─────────────┼─────────────┼─────────────┤
│      1      │      9      │     26      │
└─────────────┴─────────────┴─────────────┘


In [12]:
println("Accuracy: $(round(__evaTree.Accuracy, digits=2))")
println("Recall: $(round(__evaTree.Recall, digits=2))")
println("Precision: $(round(__evaTree.Precision, digits=2))")
println("F1: $(round(__evaTree.F1, digits=2))")
println("MCC: $(round(__evaTree.MCC, digits=2))")

Accuracy: 0.73
Recall: 0.74
Precision: 0.63
F1: 0.92
MCC: 0.46


# Neural Network

In [28]:
NNC_model = NeuralNetworkClassifier()
NNC = trainer(NNC_model, cleveland, train, test);
__evaNNC = _evaluation(NNC, cleveland, test; binary=true);

┌ Warning: The scitype of `X`, in `machine(model, X, ...)` 
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:73
┌ Info: Training Machine{NeuralNetworkClassifier{Short,…}} @382.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:317
Optimising neural net:100%[=========================] Time: 0:00:00
┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\measures\confusion_matrix.jl:83


In [29]:
__evaNNC.ConfusionMatrix

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     38      │     27      │
├─────────────┼─────────────┼─────────────┤
│      1      │     10      │     14      │
└─────────────┴─────────────┴─────────────┘


In [23]:
println("Accuracy: $(round(__evaNNC.Accuracy, digits=2))")
println("Recall: $(round(__evaNNC.Recall, digits=2))")
println("Precision: $(round(__evaNNC.Precision, digits=2))")
println("F1: $(round(__evaNNC.F1, digits=2))")
println("MCC: $(round(__evaNNC.MCC, digits=2))")

Accuracy: 0.56
Recall: 0.54
Precision: 0.37
F1: 0.81
MCC: 0.1


# AdaBoostClassifier

In [84]:
ABC_model = AdaBoostClassifier()
ABC = trainer(ABC_model, cleveland, train, test);
__evaABC = _evaluation(ABC, cleveland, test; binary=true);

┌ Warning: The scitype of `X`, in `machine(model, X, ...)` 
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:73
┌ Info: Training Machine{AdaBoostClassifier} @637.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:317
┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\measures\confusion_matrix.jl:83


In [85]:
__evaABC.ConfusionMatrix

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     41      │     14      │
├─────────────┼─────────────┼─────────────┤
│      1      │      7      │     27      │
└─────────────┴─────────────┴─────────────┘


In [86]:
println("Accuracy: $(round(__evaABC.Accuracy, digits=2))")
println("Recall: $(round(__evaABC.Recall, digits=2))")
println("Precision: $(round(__evaABC.Precision, digits=2))")
println("F1: $(round(__evaABC.F1, digits=2))")
println("MCC: $(round(__evaABC.MCC, digits=2))")

Accuracy: 0.76
Recall: 0.79
Precision: 0.66
F1: 0.91
MCC: 0.53


# KNeighborsClassifier

In [97]:
KN_model = @load KNeighborsClassifier verbosity=1
KN_model = KNeighborsClassifier()
KN = trainer(KN_model, cleveland, train, test);
__evaKN = _evaluation(KN, cleveland, test; binary=true);

import MLJScikitLearnInterface ✔
import MLJScikitLearnInterface ✔


┌ Info: Loading into module "Main": 
└ @ MLJModels C:\Users\yancf\.julia\packages\MLJModels\5DFoi\src\loading.jl:70
┌ Warning: New model type being bound to `KNeighborsClassifier3` to avoid conflict with an existing name. 
└ @ MLJModels C:\Users\yancf\.julia\packages\MLJModels\5DFoi\src\loading.jl:78
┌ Warning: The scitype of `X`, in `machine(model, X, ...)` 
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:73
┌ Info: Training Machine{KNeighborsClassifier} @909.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\machines.jl:317
┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\yancf\.julia\packages\MLJBase\8FWJ9\src\measures\confusion_matrix.jl:83


In [98]:
__evaKN.ConfusionMatrix

              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     39      │     13      │
├─────────────┼─────────────┼─────────────┤
│      1      │      9      │     28      │
└─────────────┴─────────────┴─────────────┘


In [99]:
println("Accuracy: $(round(__evaKN.Accuracy, digits=2))")
println("Recall: $(round(__evaKN.Recall, digits=2))")
println("Precision: $(round(__evaKN.Precision, digits=2))")
println("F1: $(round(__evaKN.F1, digits=2))")
println("MCC: $(round(__evaKN.MCC, digits=2))")

Accuracy: 0.75
Recall: 0.76
Precision: 0.68
F1: 0.95
MCC: 0.5
